In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
import numpy as np
import os
import sys
import time
import datetime
import glob
sys.path.append('../py/')
from s027_kfold_ods import ods_kfold
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import logger_func, get_categorical_features, get_numeric_features, reduce_mem_usage, elo_save_feature, impute_feature
try:
    if not logger:
        logger=logger_func()
except NameError:
    logger=logger_func()

from sklearn.metrics import mean_squared_error, roc_auc_score


#========================================================================
# Args
out_part = ['', 'part', 'all'][0]
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_active_month', 'index', 'personal_term', 'no_out_flg']
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())
#========================================================================

2019-02-25 15:12:59,972 utils 400 [INFO]    [logger_func] start 


In [2]:
df_hist = pd.read_csv('../input/historical_transactions.csv')
df_hist['purchase_amount_new'] = np.round(df_hist['purchase_amount'] / 0.00150265118 + 497.06, 2)

df_hist['installments'] = df_hist['installments'].map(lambda x:  
                                                    1 if x<1 else
                                                    1 if x>100 else
                                                      x
                                                     )
df_hist['authorized_flag'] = df_hist['authorized_flag'].map({'Y':1, 'N':0})
df_hist['category_1'] = df_hist['category_1'].map({'Y':1, 'N':0}) 
df_hist['yyyymm'] = df_hist['purchase_date'].map(lambda x: str(x)[:7])
df_hist['yyyymmdd'] = df_hist['purchase_date'].map(lambda x: str(x)[:10])

In [6]:
df_agg_all = df_hist.groupby([key, 'merchant_id', 'yyyymm'])['authorized_flag'].mean()
df_agg_all = df_agg_all.to_frame().reset_index()
df_agg_all.sort_values(by=[key, 'merchant_id', 'yyyymm'], ascending=False, inplace=True)
df_agg_all = utils.row_number(df=df_agg_all, level=[key, 'merchant_id'])
df_agg = df_agg_all[df_agg_all['authorized_flag']<1]
df_agg.head()

,card_id,merchant_id,yyyymm,authorized_flag,row_no
0,C_ID_fffffd5772,M_ID_fc7d7969c3,2018-02,0.200000,1
29,C_ID_fffffd5772,M_ID_445742726b,2018-02,0.333333,1
30,C_ID_fffffd5772,M_ID_445742726b,2018-01,0.750000,2
45,C_ID_ffff828181,M_ID_fc7d7969c3,2017-11,0.000000,1
61,C_ID_ffff828181,M_ID_e0abb16efe,2017-08,0.333333,1


In [6]:
level_merchant = 1
if level_merchant:
    prefix = f"272_ath"
else:
    prefix = f"271_ath"
row_list = [1,2,3]
for row in row_list:
    
    tmp = df_agg[df_agg['row_no']<=row]
    
    if row==1:
        aggs = {}
        aggs['authorized_flag'] = ['mean', 'max', 'min']
        
        tmp = tmp.groupby(key).agg(aggs)
        tmp.columns = [f"lastmonth{row}_{key}_{val}" for key in aggs.keys() for val in aggs[key]]
        last1 = tmp.copy()
        if level_merchant:
            continue
    else:
        if level_merchant:
            tmp = tmp.groupby([key, 'merchant_id']).mean().reset_index()
            tmp = tmp.groupby([key]).agg(aggs)
            tmp.columns = [f"lastmonth_{row}_mercant_{key}_{val}" for key in aggs.keys() for val in aggs[key]]
        else:
            tmp = tmp.groupby([key]).agg(aggs)
            tmp.columns = [f"lastmonth{row}_{key}_{val}" for key in aggs.keys() for val in aggs[key]]
        
        feat_cols = sorted(list(tmp.columns))
        last_cols = sorted(list(last1.columns))
        
        for last, feat in zip(last_cols, feat_cols):
            tmp[f"ratio_1_{feat}"] = last1[last] / tmp[feat]

    #========================================================================
    # Save Feature
    base = utils.read_df_pkl('../input/base_no_out*')
    base = base[[key, target]].set_index(key)
    base = base.join(tmp)
    base.fillna(-1, inplace=True)
    del tmp
    gc.collect()
    
    elo_save_feature(df_feat=base, prefix=prefix)
    print('Complete!')
    #========================================================================

100%|██████████| 1/1 [00:00<00:00,  3.55it/s]


Complete!


100%|██████████| 1/1 [00:00<00:00,  3.29it/s]


Complete!


In [7]:
tmp_list = []
row_list = [1,2,3,6]
for row in row_list:
    tmp_all = df_agg_all[df_agg_all['row_no']<=row]
    tmp = df_agg[df_agg['row_no']<=row]
    
    tmp_all['mer_cnt_all'] = 1
    tmp['auth0_mer_cnt'] = 1
    ym_cnt_all = tmp_all.groupby([key, 'merchant_id'])['mer_cnt_all'].sum()
    ym_cnt = tmp.groupby([key, 'merchant_id'])['auth0_mer_cnt'].sum()
    df_cnt = pd.concat([ym_cnt_all, ym_cnt], axis=1)
    
    df_cnt['auth0_mer_ratio'] = df_cnt['auth0_mer_cnt'] / df_cnt['mer_cnt_all']
    
    df_cnt.reset_index(inplace=True)
    feat_cols = [col for col in df_cnt.columns if col.count('mer_') and not(col.count('all'))]
    aggs = {}
    for col in feat_cols:
        if col.count('ratio'):
            aggs[col] = ['mean', 'max', 'min']
        elif col.count('cnt'):
            aggs[col] = ['sum', 'mean', 'max']
    tmp = df_cnt.groupby(key)[feat_cols].agg(aggs)
    new_cols = [f"lastmonth{row}_{key}_{val}"  for key in aggs.keys() for val in aggs[key]]
    tmp.columns = new_cols
    tmp_list.append(tmp)